# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [60]:
# import libraries
import nltk
import numpy as np
nltk.download(['punkt', 'wordnet'])
from nltk.tokenize import word_tokenize, RegexpTokenizer
from nltk.stem import WordNetLemmatizer
import pandas as pd
from sqlalchemy import create_engine
import re
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import precision_recall_fscore_support
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from sklearn.multioutput import MultiOutputClassifier
from sklearn.preprocessing import StandardScaler
import pickle
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import AdaBoostClassifier
import pickle


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [33]:
engine = create_engine('sqlite:///res.db')
df = pd.read_sql('SELECT * FROM res', engine)
X = df['message']
#droping 'child_alone' as well since its all 1's
y = df.drop(['id', 'message', 'original', 'genre','child_alone'], axis=1)

y['related']=y['related'].map(lambda x: 1 if x == 2 else x)

### 2. Write a tokenization function to process your text data

In [34]:
def tokenize(text):
    tokens = nltk.word_tokenize(text)
    lemmatizer = nltk.WordNetLemmatizer()
    
    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [35]:
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', RandomForestClassifier())
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [40]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [41]:
# train classifier
pipeline.fit(X_train, y_train)

# predict on test data
y_pred = pipeline.predict(X_test)

#X_train, X_test, y_train, y_test = train_test_split(X, y)
#pipeline.fit(X_train.as_matrix(), y_train.as_matrix())

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [44]:
def display_results(y_test, y_pred):
    labels = np.unique(y_pred)
    accuracy = (y_pred == y_test).mean()
    totalAcc = sum(accuracy)/len(accuracy)
    print(classification_report(y_test.values, y_pred, target_names=y.columns.values))
    print("----------------------------------------------------------------------")
    print("Overall: \n", totalAcc,'')


In [45]:
results = display_results(y_test, y_pred)
results

                        precision    recall  f1-score   support

               related       0.78      0.89      0.83      5008
               request       0.39      0.08      0.13      1090
                 offer       0.00      0.00      0.00        40
           aid_related       0.44      0.13      0.20      2698
          medical_help       0.00      0.00      0.00       506
      medical_products       0.20      0.01      0.01       341
     search_and_rescue       0.00      0.00      0.00       185
              security       0.00      0.00      0.00       108
              military       0.00      0.00      0.00       203
                 water       0.25      0.00      0.01       382
                  food       0.33      0.01      0.01       706
               shelter       0.09      0.00      0.00       553
              clothing       0.25      0.01      0.02        96
                 money       0.33      0.01      0.01       160
        missing_people       0.00      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [46]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x7f6124adfe18>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
               max_depth=None, max_features='auto', max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
               min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
               oob_score=False, random_state=None, verbose=0,
               warm_

In [47]:
parameters = {'clf__max_depth': [10, 20, None],
              'clf__min_samples_leaf': [1, 2, 4]
             }

cv = GridSearchCV(pipeline, param_grid=parameters, n_jobs=4, verbose=2)
cv.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)

Fitting 3 folds for each of 9 candidates, totalling 27 fits
[CV] clf__max_depth=10, clf__min_samples_leaf=1 ......................
[CV] clf__max_depth=10, clf__min_samples_leaf=1 ......................
[CV] clf__max_depth=10, clf__min_samples_leaf=1 ......................
[CV] clf__max_depth=10, clf__min_samples_leaf=2 ......................
[CV] ....... clf__max_depth=10, clf__min_samples_leaf=1, total=  45.6s
[CV] ....... clf__max_depth=10, clf__min_samples_leaf=1, total=  45.5s
[CV] clf__max_depth=10, clf__min_samples_leaf=2 ......................
[CV] clf__max_depth=10, clf__min_samples_leaf=2 ......................
[CV] ....... clf__max_depth=10, clf__min_samples_leaf=1, total=  45.5s
[CV] clf__max_depth=10, clf__min_samples_leaf=4 ......................
[CV] ....... clf__max_depth=10, clf__min_samples_leaf=2, total=  45.4s
[CV] clf__max_depth=10, clf__min_samples_leaf=4 ......................
[CV] ....... clf__max_depth=10, clf__min_samples_leaf=2, total=  45.3s
[CV] clf__max_dep

[Parallel(n_jobs=4)]: Done  27 out of  27 | elapsed:  9.4min finished


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [48]:
results2 = display_results(y_test, y_pred)
results2

                        precision    recall  f1-score   support

               related       0.78      0.89      0.83      5008
               request       0.39      0.08      0.13      1090
                 offer       0.00      0.00      0.00        40
           aid_related       0.44      0.13      0.20      2698
          medical_help       0.00      0.00      0.00       506
      medical_products       0.20      0.01      0.01       341
     search_and_rescue       0.00      0.00      0.00       185
              security       0.00      0.00      0.00       108
              military       0.00      0.00      0.00       203
                 water       0.25      0.00      0.01       382
                  food       0.33      0.01      0.01       706
               shelter       0.09      0.00      0.00       553
              clothing       0.25      0.01      0.02        96
                 money       0.33      0.01      0.01       160
        missing_people       0.00      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [50]:
cv.best_params_

{'clf__max_depth': 10, 'clf__min_samples_leaf': 4}

In [57]:
pipeline_2 = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier(),n_jobs=1))
    ])

X_train, X_test, y_train, y_test = train_test_split(X, y)
pipeline_2.fit(X_train, y_train)
y_pred = pipeline_2.predict(X_test)


In [58]:
results3 = display_results(y_test, y_pred)
results3

                        precision    recall  f1-score   support

               related       0.78      0.93      0.85      5040
               request       0.35      0.05      0.09      1127
                 offer       0.00      0.00      0.00        29
           aid_related       0.46      0.19      0.27      2721
          medical_help       0.06      0.00      0.00       492
      medical_products       0.00      0.00      0.00       325
     search_and_rescue       0.00      0.00      0.00       190
              security       0.00      0.00      0.00       111
              military       0.33      0.00      0.01       209
                 water       0.18      0.01      0.02       423
                  food       0.12      0.01      0.01       743
               shelter       0.19      0.01      0.02       593
              clothing       0.00      0.00      0.00       104
                 money       0.00      0.00      0.00       150
        missing_people       0.00      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 9. Export your model as a pickle file

In [63]:
pickle.dump(pipeline_2,open('model.pkl','wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.